In [ ]:
# MP_work_notebook

In [1]:
from alpaca.data.historical import CryptoHistoricalDataClient

# No keys required for crypto data
client = CryptoHistoricalDataClient()

In [2]:
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame

# Creating request object
request_params = CryptoBarsRequest(
  symbol_or_symbols=["BTC/USD"],
  timeframe=TimeFrame.Day,
  start="2023-01-01",
  end="2024-09-07")

In [3]:
# Retrieve daily bars for Bitcoin in a DataFrame and printing it
btc_bars = client.get_crypto_bars(request_params)

# Convert to dataframe
btc_bars.df

open       high        low  \
symbol  timestamp                                                    
BTC/USD 2023-01-01 06:00:00+00:00  16525.000  16695.310  16495.060   
        2023-01-02 06:00:00+00:00  16644.370  16779.000  16630.470   
        2023-01-03 06:00:00+00:00  16721.010  16900.000  16600.000   
        2023-01-04 06:00:00+00:00  16852.620  16984.540  16760.570   
        2023-01-05 06:00:00+00:00  16838.470  16873.280  16755.450   
...                                      ...        ...        ...   
        2024-09-02 05:00:00+00:00  57473.800  59803.450  57396.660   
        2024-09-03 05:00:00+00:00  59080.123  59320.205  55691.014   
        2024-09-04 05:00:00+00:00  56689.123  58533.075  56187.600   
        2024-09-05 05:00:00+00:00  57187.797  57267.790  55681.931   
        2024-09-06 05:00:00+00:00  56638.500  56976.450  52580.560   

                                        close       volume  trade_count  \
symbol  timestamp                                                         
BTC/USD 2023-01-01 06:00:00+00:00  16642.7800  1651.559057      34627.0   
        2023-01-02 06:00:00+00:00  16718.8200  2285.656433      41468.0   
        2023-01-03 06:00:00+00:00  16850.5900  3790.536364      64904.0   
        2023-01-04 06:00:00+00:00  16840.5000  4847.939963      81942.0   
        2023-01-05 06:00:00+00:00  16803.4700  2580.410943      57481.0   
...                                       ...          ...          ...   
        2024-09-02 05:00:00+00:00  59052.0265     0.370579         64.0   
        2024-09-03 05:00:00+00:00  56659.8115     5.096164        160.0   
        2024-09-04 05:00:00+00:00  57216.1885     2.616058        158.0   
        2024-09-05 05:00:00+00:00  56632.7325     0.818696        142.0   
        2024-09-06 05:00:00+00:00  54044.1175     5.045418        278.0   

                                           vwap  
symbol  timestamp                                
BTC/USD 2023-01-01 06:00:00+00:00  16578.560487  
        2023-01-02 06:00:00+00:00  16708.451933  
        2023-01-03 06:00:00+00:00  16722.314967  
        2023-01-04 06:00:00+00:00  16848.857844  
        2023-01-05 06:00:00+00:00  16822.708137  
...                                         ...  
        2024-09-02 05:00:00+00:00  58465.241113  
        2024-09-03 05:00:00+00:00  57243.296841  
        2024-09-04 05:00:00+00:00  56652.154248  
        2024-09-05 05:00:00+00:00  56450.003849  
        2024-09-06 05:00:00+00:00  54133.919527  

[615 rows x 7 columns]

In [4]:
request_params = CryptoBarsRequest(
  symbol_or_symbols=["ETH/USD"],
  timeframe=TimeFrame.Day,
  start="2023-01-01",
  end="2024-09-07")

In [5]:
# Retrieve daily bars for Etherium in a DataFrame and printing it
eth_bars = client.get_crypto_bars(request_params)

# Convert to dataframe
eth_bars.df

open      high        low      close  \
symbol  timestamp                                                             
ETH/USD 2023-01-01 06:00:00+00:00  1193.810  1204.390  1192.5800  1201.8100   
        2023-01-02 06:00:00+00:00  1201.950  1223.480  1200.1200  1217.4400   
        2023-01-03 06:00:00+00:00  1217.560  1254.720  1204.4800  1247.7100   
        2023-01-04 06:00:00+00:00  1247.740  1271.200  1243.3500  1253.0800   
        2023-01-05 06:00:00+00:00  1253.080  1257.530  1242.3900  1249.0100   
...                                     ...       ...        ...        ...   
        2024-09-02 05:00:00+00:00  2438.155  2561.970  2433.7025  2515.4300   
        2024-09-03 05:00:00+00:00  2516.839  2523.985  2312.8350  2376.4150   
        2024-09-04 05:00:00+00:00  2374.505  2487.485  2363.5965  2414.7245   
        2024-09-05 05:00:00+00:00  2413.750  2416.373  2348.7535  2387.1765   
        2024-09-06 05:00:00+00:00  2386.642  2404.947  2150.9035  2246.3215   

                                         volume  trade_count         vwap  
symbol  timestamp                                                          
ETH/USD 2023-01-01 06:00:00+00:00  16386.199764      21624.0  1198.344340  
        2023-01-02 06:00:00+00:00  32379.336953      33452.0  1215.523728  
        2023-01-03 06:00:00+00:00  40255.135127      47015.0  1223.886056  
        2023-01-04 06:00:00+00:00  54481.030355      66711.0  1254.162503  
        2023-01-05 06:00:00+00:00  30055.600402      38145.0  1250.447445  
...                                         ...          ...          ...  
        2024-09-02 05:00:00+00:00     11.006654         34.0  2445.510841  
        2024-09-03 05:00:00+00:00     47.350061         72.0  2430.408477  
        2024-09-04 05:00:00+00:00     53.181289         50.0  2419.041750  
        2024-09-05 05:00:00+00:00      7.983776         69.0  2381.809768  
        2024-09-06 05:00:00+00:00     44.756517        127.0  2271.649308  

[615 rows x 7 columns]

In [6]:
request_params = CryptoBarsRequest(
  symbol_or_symbols=["DOGE/USD"],
  timeframe=TimeFrame.Day,
  start="2023-01-01",
  end="2024-09-07")

In [7]:
# Retrieve daily bars for Dogecoin in a DataFrame and printing it
doge_bars = client.get_crypto_bars(request_params)

# Convert to dataframe
doge_bars.df

open      high       low     close  \
symbol   timestamp                                                           
DOGE/USD 2023-01-01 06:00:00+00:00  0.069400  0.070800  0.069000  0.070200   
         2023-01-02 06:00:00+00:00  0.070200  0.073500  0.070000  0.071900   
         2023-01-03 06:00:00+00:00  0.072000  0.073030  0.069500  0.072300   
         2023-01-04 06:00:00+00:00  0.072300  0.075000  0.071600  0.074900   
         2023-01-05 06:00:00+00:00  0.074700  0.075200  0.070600  0.071000   
...                                      ...       ...       ...       ...   
         2024-09-02 05:00:00+00:00  0.094183  0.100777  0.094130  0.099171   
         2024-09-03 05:00:00+00:00  0.099140  0.100500  0.092022  0.095877   
         2024-09-04 05:00:00+00:00  0.095769  0.099475  0.094501  0.096860   
         2024-09-05 05:00:00+00:00  0.096794  0.099386  0.096295  0.098492   
         2024-09-06 05:00:00+00:00  0.098440  0.098910  0.088967  0.092460   

                                          volume  trade_count      vwap  
symbol   timestamp                                                       
DOGE/USD 2023-01-01 06:00:00+00:00  1.474223e+07       1231.0  0.069857  
         2023-01-02 06:00:00+00:00  2.981534e+07       2044.0  0.071841  
         2023-01-03 06:00:00+00:00  2.361700e+07       2062.0  0.071140  
         2023-01-04 06:00:00+00:00  2.635153e+07       2312.0  0.072905  
         2023-01-05 06:00:00+00:00  3.362629e+07       2341.0  0.072438  
...                                          ...          ...       ...  
         2024-09-02 05:00:00+00:00  2.365866e+05         29.0  0.099701  
         2024-09-03 05:00:00+00:00  2.522752e+04         26.0  0.096683  
         2024-09-04 05:00:00+00:00  3.222848e+04         22.0  0.096011  
         2024-09-05 05:00:00+00:00  1.465725e+04         27.0  0.098316  
         2024-09-06 05:00:00+00:00  2.800621e+04         31.0  0.092581  

[615 rows x 7 columns]

In [8]:
request_params = CryptoBarsRequest(
  symbol_or_symbols=["LTC/USD"],
  timeframe=TimeFrame.Day,
  start="2023-01-01",
  end="2024-09-07")

In [9]:
# Retrieve daily bars for Litecoin in a DataFrame and printing it
ltc_bars = client.get_crypto_bars(request_params)

# Convert to dataframe
ltc_bars.df

open     high      low     close  \
symbol  timestamp                                                        
LTC/USD 2023-01-01 06:00:00+00:00  69.3500  71.4400  69.1500  70.54000   
        2023-01-02 06:00:00+00:00  70.5700  77.6900  70.3600  75.73000   
        2023-01-03 06:00:00+00:00  75.7000  77.9300  73.6700  76.59000   
        2023-01-04 06:00:00+00:00  76.5900  77.1700  74.3100  75.79000   
        2023-01-05 06:00:00+00:00  75.8300  75.8300  73.5700  73.94000   
...                                    ...      ...      ...       ...   
        2024-09-02 05:00:00+00:00  63.0910  66.2720  62.5910  65.76985   
        2024-09-03 05:00:00+00:00  65.7517  66.3070  62.1850  65.78500   
        2024-09-04 05:00:00+00:00  65.7660  66.9865  64.1225  65.40995   
        2024-09-05 05:00:00+00:00  65.3450  68.4993  64.4150  66.34450   
        2024-09-06 05:00:00+00:00  66.2540  66.2540  61.1885  63.90055   

                                         volume  trade_count       vwap  
symbol  timestamp                                                        
LTC/USD 2023-01-01 06:00:00+00:00  11841.135572       1610.0  70.529794  
        2023-01-02 06:00:00+00:00  26633.145245       4135.0  74.949142  
        2023-01-03 06:00:00+00:00  25448.265751       3490.0  75.996053  
        2023-01-04 06:00:00+00:00  16161.357606       1979.0  75.787561  
        2023-01-05 06:00:00+00:00   8598.549829       1467.0  74.485081  
...                                         ...          ...        ...  
        2024-09-02 05:00:00+00:00     16.324382         12.0  65.923997  
        2024-09-03 05:00:00+00:00      6.805679         20.0  65.799070  
        2024-09-04 05:00:00+00:00     28.797788         17.0  64.958421  
        2024-09-05 05:00:00+00:00    143.918686         33.0  67.495107  
        2024-09-06 05:00:00+00:00      4.592779         20.0  63.828826  

[615 rows x 7 columns]

In [10]:
import alpaca_trade_api as tradeapi
import pandas as pd
import pandas_ta as ta
# import mplfinance as mpf

#Loading Alpaca API keys
API_Key = "PK688IQ8GKXYNUE4M7YX"
API_secret_key = "LKfuASkhgHQm6xnD6em7CHrZbPoB8Nt7Pabo0jPZ"
API_base_url = "https://paper-api.alpaca.markets"
api = tradeapi.REST(API_Key, API_secret_key, API_base_url, api_version='v2')

In [12]:
def fetch_data(symbol, start, end, timeframe='1D'):
    # Fetching bar data from Alpaca
    barset = api.get_bars(symbol, timeframe, start=start, end=end)

    # Convert the barset to a DataFrame
    bars_df = pd.DataFrame([{
        'time': bar.t.strftime('%Y-%m-%d %H:%M:%S'),  # Converting timestamp to string
        'open': bar.o,
        'high': bar.h,
        'low': bar.l,
        'close': bar.c,
        'volume': bar.v
    } for bar in barset])

    return bars_df


# Fetching NVDA data and returning as a DataFrame
nvda_df = fetch_data('NVDA', '2023-01-01', '2024-09-01')
nvda_df


,time,open,high,low,close,volume
0,2023-01-03 00:00:00,148.510,149.96,140.9600,143.15,41810749
1,2023-01-04 00:00:00,145.670,148.53,142.4100,147.49,45047174
2,2023-01-05 00:00:00,144.910,145.64,141.4795,142.65,41702951
3,2023-01-06 00:00:00,144.740,150.10,140.3400,148.59,43123117
4,2023-01-09 00:00:00,152.840,160.56,151.4100,156.28,52588232
...,...,...,...,...,...,...
413,2024-08-26 00:00:00,129.570,131.26,124.3700,126.46,331964672
414,2024-08-27 00:00:00,125.050,129.20,123.8800,128.30,303134573
415,2024-08-28 00:00:00,128.120,128.33,122.6400,125.61,448101115
416,2024-08-29 00:00:00,121.355,124.43,116.7100,117.59,453023305


In [13]:
amd_df = fetch_data('AMD', '2023-01-01', '2024-09-01')
amd_df

,time,open,high,low,close,volume
0,2023-01-03 00:00:00,65.998,66.88,63.5900,64.02,48737338
1,2023-01-04 00:00:00,65.000,65.79,63.3100,64.66,49627944
2,2023-01-05 00:00:00,64.150,64.35,62.3000,62.33,49271933
3,2023-01-06 00:00:00,63.145,64.30,60.0500,63.96,73718225
4,2023-01-09 00:00:00,66.220,69.32,65.6700,67.24,73012565
...,...,...,...,...,...,...
413,2024-08-26 00:00:00,154.700,158.28,148.9101,149.99,49893259
414,2024-08-27 00:00:00,150.130,151.70,148.4400,150.50,35102650
415,2024-08-28 00:00:00,149.400,150.43,144.7200,146.36,34075823
416,2024-08-29 00:00:00,146.590,149.49,144.4700,145.49,31602113


In [15]:
tsm_df = fetch_data('TSM', '2023-01-01', '2024-09-01')
tsm_df

,time,open,high,low,close,volume
0,2023-01-03 00:00:00,75.850,75.970,73.770,74.03,9276191
1,2023-01-04 00:00:00,74.990,76.610,74.470,76.32,12316386
2,2023-01-05 00:00:00,76.000,76.440,75.340,75.73,10039194
3,2023-01-06 00:00:00,76.500,78.739,76.150,78.07,13734103
4,2023-01-09 00:00:00,80.560,81.215,80.055,80.31,22247607
...,...,...,...,...,...,...
413,2024-08-26 00:00:00,170.380,171.050,167.820,169.07,9611328
414,2024-08-27 00:00:00,166.420,170.580,165.040,170.55,9186041
415,2024-08-28 00:00:00,171.320,172.465,166.200,169.20,10581371
416,2024-08-29 00:00:00,168.650,175.420,167.510,169.15,13213194


In [16]:
intc_df = fetch_data('INTC', '2023-01-01', '2024-09-01')
intc_df

,time,open,high,low,close,volume
0,2023-01-03 00:00:00,27.045,27.070,26.37,26.73,34639049
1,2023-01-04 00:00:00,27.410,27.920,27.16,27.68,43232398
2,2023-01-05 00:00:00,27.435,27.779,27.38,27.56,43178055
3,2023-01-06 00:00:00,27.930,28.830,27.38,28.73,37562035
4,2023-01-09 00:00:00,28.840,29.865,28.84,29.31,43778291
...,...,...,...,...,...,...
413,2024-08-26 00:00:00,20.530,20.560,19.95,20.13,58534794
414,2024-08-27 00:00:00,20.130,20.260,19.84,20.07,43862017
415,2024-08-28 00:00:00,19.980,20.105,19.51,19.61,50144885
416,2024-08-29 00:00:00,19.760,20.600,19.75,20.13,64529661


In [22]:
snp_500_df = fetch_data('SPY', '2023-01-01', '2024-09-01')
snp_500_df

,time,open,high,low,close,volume
0,2023-01-03 00:00:00,384.37,386.43,377.8310,380.82,74850731
1,2023-01-04 00:00:00,383.18,385.88,380.0000,383.76,85934098
2,2023-01-05 00:00:00,381.72,381.84,378.7600,379.38,76970456
3,2023-01-06 00:00:00,382.61,389.25,379.4127,388.08,104026233
4,2023-01-09 00:00:00,390.37,393.70,387.6700,387.86,73978071
...,...,...,...,...,...,...
413,2024-08-26 00:00:00,563.18,563.91,559.0500,560.79,35788609
414,2024-08-27 00:00:00,559.49,562.06,558.3200,561.56,32693898
415,2024-08-28 00:00:00,561.21,561.65,555.0400,558.30,41066024
416,2024-08-29 00:00:00,560.31,563.68,557.1800,558.35,38715176
